In [1]:
from __future__ import print_function
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

cam=cv2.VideoCapture(0)

while cam.isOpened:
    ret,frame=cam.read()
    cv2.imshow("video",frame)
    c=cv2.waitKey(1)
    if(c==27):
        break
cam.release()
cv2.destroyAllWindows()

In [2]:
from __future__ import print_function
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

def order_point_old(pts):
    rect=np.zeros((4,2),dtype="float32")
    
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    
    return rect

cam=cv2.VideoCapture(0)

while cam.isOpened:
    ret,frame=cam.read()
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    blurred=cv2.GaussianBlur(gray,(17,17),0)
    edged=cv2.Canny(gray,50,100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    
    #find contour
    cnts=cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,
                         cv2.CHAIN_APPROX_SIMPLE)
    cnts=imutils.grab_contours(cnts)
    
    (cnts,_)=contours.sort_contours(cnts)
    colors = ((0, 0, 255), (240, 0, 159), (255, 0, 0), (255, 255, 0))
    
    for (i,c) in enumerate(cnts):
        if cv2.contourArea(c)<150 or cv2.contourArea(c)>700:
            continue
        
        #compute the rotation of object
        box=cv2.minAreaRect(c)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")#this is giving orignal co-ordinates point
        cv2.drawContours(frame, [box], -1, (0, 255, 0), 2)
        
#         print("Objecwt #{}:".format(i + 1))
#         print(box)
        
        rect=order_point_old(box)
        rect = perspective.order_points(box)
        
        
#         print("rect=",rect)
        for ((x, y), color) in zip(rect, colors):
            cv2.circle(frame, (int(x), int(y)), 5, color, -1)
        
        cv2.putText(frame, "Object #{}".format(i + 1),
            (int(rect[0][0] - 15), int(rect[0][1] - 15)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 2)
        
    cv2.imshow("edged",edged)
    cv2.imshow("video",frame)
    ca=cv2.waitKey(1)
    if(ca==27):
        break
cam.release()
cv2.destroyAllWindows()

In [6]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import numpy as np
import imutils
import cv2

def mid(ptA,ptB):
    return ((ptA[0]+ptB[0])*0.5,(ptA[1]+ptB[1])*0.5)
cam=cv2.VideoCapture(0)

while cam.isOpened:
    ret,frame=cam.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(11,11),0)
    edged = cv2.Canny(gray, 50, 100)
    edged = cv2.dilate(edged, None, iterations=1)
    edged = cv2.erode(edged, None, iterations=1)
    
    #finding contour
    cnts=cv2.findContours(edged.copy(),cv2.RETR_EXTERNAL,
                          cv2.CHAIN_APPROX_SIMPLE)
    cnts=imutils.grab_contours(cnts)
    
    (cnts,_)=contours.sort_contours(cnts)
    pixelPerMetric=None
    
    for c in cnts:
        if(cv2.contourArea(c)<100):
            continue
        orig=frame.copy()
        box=cv2.minAreaRect(c)
        box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
        box = np.array(box, dtype="int")
        
        box=perspective.order_points(box)
        cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)
        
        for (x,y) in box:
            cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)    
        
        (tl,tr,bl,br)=box
        (topx,topy)=mid(tl,tr)
        (bottomx,bottomy)=mid(bl,br)
        (leftx,lefty)=mid(tl,bl)
        (rightx,righty)=mid(tr,br)
        
        cv2.circle(orig, (int(topx), int(topy)), 5, (255, 0, 0), -1)
        cv2.circle(orig, (int(bottomx), int(bottomy)), 5, (255, 0, 0), -1)
        cv2.circle(orig, (int(leftx), int(lefty)), 5, (255, 0, 0), -1)
        cv2.circle(orig, (int(rightx), int(righty)), 5, (255, 0, 0), -1)
        
        cv2.line(orig, (int(topx), int(topy)), (int(bottomx), int(bottomy)),
                        (255, 0, 255), 2)
        cv2.line(orig, (int(leftx), int(lefty)), (int(rightx), int(righty)),
                        (255, 0, 255), 2)
        #calcute distance between distance between two points
        dA=dist.euclidean((topx,topy),(bottomx,bottomy))
        dB=dist.euclidean((leftx,lefty),(rightx,righty))
            
        if(pixelPerMetric is None):
            # width of any refernce  object is given, i gave 6in and 3in
            pixelPerMetric=dB/6

        dimA=dA/pixelPerMetric
        dimB=dB/pixelPerMetric

        cv2.putText(orig, "{:.1f}in".format(dimA),
                    (int(topx - 15), int(topy - 10)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
        cv2.putText(orig, "{:.1f}in".format(dimB),
                    (int(rightx + 10), int(righty)),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.65, (255, 255, 255), 2)
        
    cv2.imshow("video",orig)
    c=cv2.waitKey(1)
    if(c==27):
        break
cam.release()
cv2.destroyAllWindows()

ZeroDivisionError: float division by zero